# 第四节 学习Memory及其应用实践

### 课程笔记大纲

1. 记忆管理介绍
2. LangChain的记忆类型
3. ConversationBufferMemory 记忆
4. ConversationBufferWindowMemory 记忆
5. ConversationTokenBufferMemory 记忆
6. ConversationSummaryBufferMemory 记忆

## 记忆管理介绍


LangChain记忆管理是提供了管理记忆的工具，提供了多种复杂的内存管理选项，使用户能够灵活地控制对话历史的存储和管理方式。

在介绍不同的记忆组件的时候，我们先使用人工方法推入2-3轮对话记录，通过给记忆组件设置不同的参数，打印最后记忆组件记住的内容，对比各种类型记忆组件。最后我们都会使用ConversationChain会话链，将记忆组件赋值给会话链，构造出具有记忆的会话链。

### 环境安装配置

首先，为了能够顺利进行开发工作，需要确保机器上安装了相应的Python包。开发者可以通过以下命令轻松完成安装：

In [ ]:
%pip install --upgrade --quiet  openai langchain-openai langchain

导入OpenAI模块,设置好开发密钥。

In [ ]:
import os
API_SECRET_KEY = "填写【个人资料】中获取的最新token"
BASE_URL = "https://www.ai360labs.com/openai/v1/"

os.environ["OPENAI_API_KEY"] = API_SECRET_KEY
os.environ["OPENAI_BASE_URL"] = BASE_URL

In [ ]:

import openai
from langchain_openai import ChatOpenAI,OpenAI

## LangChain的记忆类型 ConversationBufferMemory

ConversationBufferMemory通过将每个对话交换保存为单独的记录或条目来存储对话历史。

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory


LangChain 记忆管理类的使用方式：

1. 实例化该类
2. 实例化一个会话链 ConversationChain ， 设置 memory 属性为记忆类的实例。

In [ ]:
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    verbose=True,
    memory = memory
)

依次进行以下三个问题，先告诉LLM我的名字，然后岔开话题问中国北京的首都是哪里，最后询问是否记得我的名字。

In [ ]:
conversation.predict(input="你好！我的名字是李特丽")

In [ ]:
conversation.predict(input="中国的首都是哪里?")

In [ ]:
conversation.predict(input="我的名字是什么?")

In [ ]:
memory.load_memory_variables({})

为了更清晰地对比，这次我们并不使用ConversationChain会话链，我们重新设置一个记忆组件。

ConversationBufferMemory的工作机制是保存单独的条目来存储对话历史。

我们通过save_context方法，给记忆组件添加一些对话记录。

In [ ]:
memory = ConversationBufferMemory()

使用save_context可以向记忆组件添加新的记录。

In [ ]:
memory.save_context({"input": "Hi"}, 
                    {"output": "What's up"})

打印记忆组件，查看里面的存储记忆。

In [ ]:
print(memory.buffer)

In [ ]:
memory.load_memory_variables({})

再次添加新的记录。

In [ ]:
memory.save_context({"input": "Not much, just hanging"}, 
                    {"output": "Cool"})

通过查看和对比，感知 ConversationBufferMemory 类型的记忆组件的特点，便于你更快掌握这个记忆组件。

In [ ]:
memory.load_memory_variables({})

## ConversationBufferWindowMemory

ConversationBufferWindowMemory固定存储最近的一段时间内的对话交换。

   - 它会限制存储的对话交换数量，而不是存储整个对话历史。
   - 每当新的对话交换发生时，系统会将其添加到内存中，并自动删除最早的对话交换，以保持固定数量的对话交换。

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

设置仅仅保存最近一次的记录(k=1)。你可以随意修改K值，测试记忆组件的效果。

In [ ]:
memory = ConversationBufferWindowMemory(k=1) 

模拟两条对话记录。

In [ ]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})

查看记忆组件记住了什么？

In [ ]:
memory.load_memory_variables({})

现在，我们使用ConversationChain会话链，完整地构造 ConversationBufferWindowMemory 记忆组件。

In [ ]:
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferWindowMemory(k=1) # just remember 1 conversational exchange
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=False
)

依次进行以下三个问题，先告诉LLM我的名字，然后岔开话题问中国北京的首都是哪里，最后询问是否记得我的名字。

In [ ]:
conversation.predict(input="你好！我叫李特丽")

In [ ]:
conversation.predict(input="中国的首都是哪里?")

与上一个案例不同的是，这一次它忘记了我的名字,因为我们设置的是仅仅记住最后一轮的对话记录(memory = ConversationBufferWindowMemory(k=1) )

In [ ]:
conversation.predict(input="我叫什么名字?")

打印后看到它仅仅记住了最后一轮对话。

In [ ]:
memory.load_memory_variables({})

## ConversationTokenBufferMemory

按照Token数形式存储对话历史：
   - ConversationSummaryBufferMemory不直接存储完整的对话内容，而是使用限制token数量的形式存储对话历史。
   - 它只提取出对话中的最后的对话记录，切割出的聊天记录满足 max_token_limit 所限制的token数。

计算记忆组件的记忆内容时，按照token数量进行存储记忆。token数量需要使用到 tiktoken 模块进行分词，所以需要提前下载。

In [ ]:
#!pip install tiktoken

进行模块导入和ChatOpenAI初始化。

In [ ]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI
llm = ChatOpenAI(temperature=0.0)

为了方便测试效果，我们采用手动添加聊天记录。设置记忆组件的 max_token_limit 为30。 也就是说仅仅记忆30个token。

In [ ]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=30)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

看看这种类型的记忆组件保存的结果。

我们可以看到，记忆组件保存了30个token的内容。

In [ ]:
memory.load_memory_variables({})

## ConversationSummaryMemory

使用摘要形式存储对话历史：
   - ConversationSummaryBufferMemory不直接存储完整的对话内容，而是使用摘要形式存储对话历史。
   - 它会对每个对话交换进行摘要处理，提取出对话中的关键信息和重要内容，并以摘要形式保存。


导入ConversationSummaryBufferMemory记忆组件。

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory


手工推入多条聊天记录，最后一条输入大量的文字内容 （变量 schedule）。

In [ ]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

查看最终的记忆结果。虽然我们输入了非常长的对话记录，但是输出的结果被压缩了。

In [ ]:
memory.load_memory_variables({})

现在，我们使用ConversationChain完整地构造 ConversationBufferWindowMemory 记忆组件。

In [ ]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

为了分清楚这些记忆组件的差别，请多测试，通常来说，改变参数值，手动添加聊天记录都是测试的好方法。

In [ ]:
conversation.predict(input="What would be a good demo to show?")

In [ ]:
memory.load_memory_variables({})